In [1]:
 import pandas as pd

In [2]:
data = pd.read_csv('/content/all_data.csv')

In [3]:
data.columns

Index(['problem', 'level', 'type', 'modified_problem', 'no_of_equations',
       'no_of_variables', 'has_exp_or_mod', 'has_logarithm', 'has_fraction',
       'has_eq_or_neq', 'sentence_count', 'word_count', 'words_per_sentence',
       'average_word_length', 'large_words', 'has_repeated_large_words',
       'CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC',
       'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT',
       'QUANTITY', 'TIME', 'WORK_OF_ART', 'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ',
       'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT',
       'SCONJ', 'SYM', 'VERB', 'X'],
      dtype='object')

In [4]:
data.type.value_counts()

Algebra                 1736
Intermediate Algebra    1295
Prealgebra              1205
Name: type, dtype: int64

In [5]:
size_mapping = {"Prealgebra": 1, "Intermediate Algebra": 2, "Algebra": 3}
data['type'] = data['type'].replace(size_mapping)

In [6]:
data.drop(columns = ['problem', 'modified_problem'], inplace = True)

In [7]:
for i in range(data.shape[0]):
  data['level'][i] = data['level'][i][6]

<ipython-input-7-992cb1e16ca9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['level'][i] = data['level'][i][6]


In [8]:
X = data.drop(columns = ['level'])
y = data['level']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2, stratify = y)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluation(y_train_pred, y_test_pred, y_train, y_test):
  print("train_evaluation:\n")
  print(classification_report(y_train_pred, y_train))
  print(confusion_matrix(y_train_pred, y_train))
  print("\ntest_evaluation:\n")
  print(classification_report(y_test_pred, y_test))
  print(confusion_matrix(y_test_pred, y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
rfc=RandomForestClassifier(n_estimators=25)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],                     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],            # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 3],              # Minimum number of samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],    # Number of features to consider at each split
    'criterion': ['gini', 'entropy', 'log_loss'],
    'oob_score': [True, False],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Perform grid search to find the best combination of parameters
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=25),
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [3, 5, 7],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 5, 10],
                         'oob_score': [True, False]})

In [ ]:
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Best Parameters:  {'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'oob_score': True}
Best Score:  0.3550755327817065


In [ ]:
y_test_pred = grid_search.predict(X_test)
y_train_pred = grid_search.predict(X_train)
evaluation(y_train_pred, y_test_pred, y_train, y_test)

train_evaluation:

              precision    recall  f1-score   support

     Level 1       0.84      0.37      0.52       659
     Level 2       0.49      0.51      0.50       638
     Level 3       0.36      0.57      0.45       481
     Level 4       0.38      0.63      0.48       472
     Level 5       0.72      0.57      0.64      1138

    accuracy                           0.53      3388
   macro avg       0.56      0.53      0.51      3388
weighted avg       0.60      0.53      0.54      3388

[[245 155 132  86  41]
 [ 15 324 107 112  80]
 [ 15  63 275  67  61]
 [  8  36  63 299  66]
 [  7  87 177 222 645]]

test_evaluation:

              precision    recall  f1-score   support

     Level 1       0.51      0.23      0.31       163
     Level 2       0.31      0.35      0.33       150
     Level 3       0.20      0.36      0.25       104
     Level 4       0.17      0.26      0.21       131
     Level 5       0.63      0.47      0.54       300

    accuracy                   

In [ ]:
rfc=RandomForestClassifier(n_estimators=25, 'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'oob_score': True)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
import numpy as np

importances = best_model.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print feature importance rankings
print("Feature Importance Rankings:")
for i, feature in enumerate(X.columns[indices]):
    print(f"{i + 1}. {feature}: {importances[indices[i]] * 100}")

Feature Importance Rankings:
1. no_of_variables: 10.534884553751588
2. large_words: 7.820602246904629
3. NOUN: 7.405840275163294
4. no_of_equations: 6.471086572939427
5. word_count: 6.019727847558337
6. words_per_sentence: 5.705464884486477
7. ADJ: 5.100419531571054
8. VERB: 4.672798851770758
9. ADP: 4.397426298980272
10. PUNCT: 4.209289248108592
11. DET: 4.181634979217914
12. average_word_length: 2.831233886918099
13. AUX: 2.4043936355762168
14. has_exp_or_mod: 2.308282214723872
15. type: 2.0484293303012597
16. CARDINAL: 2.0118145573026847
17. has_eq_or_neq: 1.9349423182344445
18. SCONJ: 1.8690192016514677
19. PROPN: 1.846550064323375
20. PRON: 1.828751980744522
21. NUM: 1.7852905546217077
22. CCONJ: 1.6070928821607309
23. ADV: 1.5550943177756067
24. has_fraction: 1.2523441360010943
25. sentence_count: 1.1703039784604345
26. PERSON: 0.7909236384869286
27. PART: 0.7455330119248139
28. DATE: 0.6799222850351582
29. ORG: 0.6725670103613203
30. SYM: 0.6379011393764117
31. QUANTITY: 0.50515

##XGBoost

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4236 entries, 0 to 4235
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   level                     4236 non-null   object
 1   type                      4236 non-null   int64 
 2   no_of_equations           4236 non-null   int64 
 3   no_of_variables           4236 non-null   int64 
 4   has_exp_or_mod            4236 non-null   bool  
 5   has_logarithm             4236 non-null   bool  
 6   has_fraction              4236 non-null   bool  
 7   has_eq_or_neq             4236 non-null   bool  
 8   sentence_count            4236 non-null   int64 
 9   word_count                4236 non-null   int64 
 10  words_per_sentence        4236 non-null   int64 
 11  average_word_length       4236 non-null   int64 
 12  large_words               4236 non-null   int64 
 13  has_repeated_large_words  4236 non-null   bool  
 14  CARDINAL                

In [ ]:
data.columns

Index(['level', 'type', 'no_of_equations', 'no_of_variables', 'has_exp_or_mod',
       'has_logarithm', 'has_fraction', 'has_eq_or_neq', 'sentence_count',
       'word_count', 'words_per_sentence', 'average_word_length',
       'large_words', 'has_repeated_large_words', 'CARDINAL', 'DATE', 'EVENT',
       'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL',
       'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME',
       'WORK_OF_ART', 'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET',
       'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'],
      dtype='object')

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load your dataset and split it into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
dtest = xgb.DMatrix(X_test)

# Set the parameters for XGBoost
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': 5,
    'eval_metric': 'logloss'
}

# Train the XGBoost model
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
preds = model.predict(dtest)
predictions = [round(value) for value in preds]

# Decode the predicted labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 38.33%


In [ ]:
preds = model.predict(dtest)
predictions = [round(value) for value in preds]

# Decode the predicted labels
predicted_test_labels = label_encoder.inverse_transform(predictions)

preds = model.predict(dtrain)
predictions = [round(value) for value in preds]

# Decode the predicted labels
predicted_train_labels = label_encoder.inverse_transform(predictions)

In [ ]:
evaluation(predicted_train_labels, predicted_test_labels, y_train, y_test)

train_evaluation:

              precision    recall  f1-score   support

           1       0.27      0.58      0.36       133
           2       0.60      0.44      0.51       899
           3       0.35      0.47      0.40       565
           4       0.35      0.49      0.41       562
           5       0.71      0.52      0.60      1229

    accuracy                           0.49      3388
   macro avg       0.45      0.50      0.46      3388
weighted avg       0.54      0.49      0.50      3388

[[ 77  23  15  11   7]
 [119 397 183 128  72]
 [ 43  74 263 100  85]
 [ 26  65 104 275  92]
 [ 25 106 189 272 637]]

test_evaluation:

              precision    recall  f1-score   support

           1       0.12      0.36      0.19        25
           2       0.49      0.35      0.41       236
           3       0.28      0.36      0.31       143
           4       0.16      0.28      0.21       115
           5       0.67      0.46      0.54       329

    accuracy                   

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit the label encoder to y_train and transform y_train to integers
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

## Hyper parameter Tuning

In [ ]:

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# Set the parameters for XGBoost
parameters = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 500, 1000],
    'eta': [0.1],
    'objective': ['multi:softmax', 'multi:softprob'],
    'num_class': [5],
    'eval_metric': ['mlogloss', 'merror', 'auc']
}

# Create the XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Perform grid search using GridSearchCV
grid_search = GridSearchCV(xgb_model, parameters, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

# Get the best parameters and best score from the grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters: ", best_params)
print("Best Score: ", best_score)

# Train the XGBoost model using the best parameters
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)

# Make predictions on the test set
preds = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [36]:

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
dtest = xgb.DMatrix(X_test)

# Set the parameters for XGBoost
params = {
    'max_depth': 6,
    'learning_rate': 0.1,
    'eta': 0.01,
    'objective': 'multi:softmax',
    'num_class': 5,
    'eval_metric': 'merror'
}

# Train the XGBoost model
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
preds = model.predict(dtest)
predictions = [round(value) for value in preds]

# Decode the predicted labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 39.15%


In [37]:
preds = model.predict(dtest)
predictions = [round(value) for value in preds]

# Decode the predicted labels
predicted_test_labels = label_encoder.inverse_transform(predictions)

preds = model.predict(dtrain)
predictions = [round(value) for value in preds]

# Decode the predicted labels
predicted_train_labels = label_encoder.inverse_transform(predictions)

evaluation(predicted_train_labels, predicted_test_labels, y_train, y_test)

train_evaluation:

              precision    recall  f1-score   support

           1       0.70      0.79      0.74       256
           2       0.78      0.73      0.75       706
           3       0.73      0.71      0.72       772
           4       0.73      0.82      0.78       703
           5       0.89      0.83      0.86       951

    accuracy                           0.78      3388
   macro avg       0.77      0.78      0.77      3388
weighted avg       0.78      0.78      0.78      3388

[[203  26  13   8   6]
 [ 41 517  76  55  17]
 [ 32  56 551  85  48]
 [  8  35  54 577  29]
 [  6  31  60  61 793]]

test_evaluation:

              precision    recall  f1-score   support

           1       0.17      0.29      0.21        41
           2       0.41      0.37      0.39       182
           3       0.37      0.38      0.37       186
           4       0.25      0.32      0.28       154
           5       0.59      0.46      0.52       285

    accuracy                   